Data cleaning and EDA of the Nashville Housing data using SQL for learning purpose

1\. Covert date format

In [46]:
Update Nashville
set SaleDate = convert(date,SaleDate)

Update Nashville
set ParcelID = convert(NVARCHAR(255),ParcelID)

Update Nashville
set SoldAsVacant = convert(varbinary,SoldAsVacant)

Update Nashville
set SalePrice = convert(money,SalePrice)

select top(5) *
from Nashville




(56477 rows affected)

(56477 rows affected)

(56477 rows affected)

(56477 rows affected)

(5 rows affected)

Total execution time: 00:00:01.979

UniqueID,ParcelID,LandUse,PropertyAddress,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,OwnerAddress,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath
2045,007 00 0 125.00,SINGLE FAMILY,"1808 FOX CHASE DR, GOODLETTSVILLE",2013-04-09,240000.00,20130412-0036474,No,"FRAZIER, CYRENTHA LYNETTE","1808 FOX CHASE DR, GOODLETTSVILLE, TN",2.3,GENERAL SERVICES DISTRICT,50000,168200,235700,1986,3,3,0
16918,007 00 0 130.00,SINGLE FAMILY,"1832 FOX CHASE DR, GOODLETTSVILLE",2014-06-10,366000.00,20140619-0053768,No,"BONER, CHARLES & LESLIE","1832 FOX CHASE DR, GOODLETTSVILLE, TN",3.5,GENERAL SERVICES DISTRICT,50000,264100,319000,1998,3,3,2
54582,007 00 0 138.00,SINGLE FAMILY,"1864 FOX CHASE DR, GOODLETTSVILLE",2016-09-26,435000.00,20160927-0101718,No,"WILSON, JAMES E. & JOANNE","1864 FOX CHASE DR, GOODLETTSVILLE, TN",2.9,GENERAL SERVICES DISTRICT,50000,216200,298000,1987,4,3,0
43070,007 00 0 143.00,SINGLE FAMILY,"1853 FOX CHASE DR, GOODLETTSVILLE",2016-01-29,255000.00,20160129-0008913,No,"BAKER, JAY K. & SUSAN E.","1853 FOX CHASE DR, GOODLETTSVILLE, TN",2.6,GENERAL SERVICES DISTRICT,50000,147300,197300,1985,3,3,0
22714,007 00 0 149.00,SINGLE FAMILY,"1829 FOX CHASE DR, GOODLETTSVILLE",2014-10-10,278000.00,20141015-0095255,No,"POST, CHRISTOPHER M. & SAMANTHA C.","1829 FOX CHASE DR, GOODLETTSVILLE, TN",2,GENERAL SERVICES DISTRICT,50000,152300,202300,1984,4,3,0


2\. populate propertyAddress with null values

In [42]:
select TableA.UniqueID,TableB.uniqueID,TableA.ParcelID,TableA.PropertyAddress,TableB.PropertyAddress
from Nashville TableA
join NaShville TableB
on TableA.UniqueID <> TableB.UniqueID
AND TableA.ParcelID = TableB.ParcelID
WHERE TableA.PropertyAddress IS NULL


UPDATE TABLEA
SET propertyAddress = isnull(TABLEA.propertyAddress,TABLEB.propertyAddress)
from Nashville TableA
join NaShville TableB
on TableA.UniqueID <> TableB.UniqueID
AND TableA.ParcelID = TableB.ParcelID
where TableA.PropertyAddress is null




(0 rows affected)

(0 rows affected)

Total execution time: 00:00:00.555

UniqueID,uniqueID,ParcelID,PropertyAddress,PropertyAddress


2b. populate OwnerAddress with null values if applicable

In [105]:
select TableA.UniqueID,TableB.uniqueID,TableA.ParcelID,TableA.OwnerAddress,TableB.OwnerAddress
from Nashville TableA
join NaShville TableB
on TableA.UniqueID <> TableB.UniqueID
AND TableA.ParcelID = TableB.ParcelID
WHERE TableA.OwnerAddress IS NULL AND TableB.OwnerAddress is not null

UPDATE TABLEA
SET OwnerAddress = isnull(TABLEA.OwnerAddress,TABLEB.OwnerAddress)
from Nashville TableA
join NaShville TableB
on TableA.UniqueID <> TableB.UniqueID
AND TableA.ParcelID = TableB.ParcelID
where TableA.OwnerAddress is null


(0 rows affected)

(8343 rows affected)

Total execution time: 00:00:00.603

UniqueID,uniqueID,ParcelID,OwnerAddress,OwnerAddress


3\. substring the propertyAddress column

In [56]:
ALTER TABLE Nashville
Add PropertyCityAddress NVARCHAR(255);


: Msg 2705, Level 16, State 4, Line 1
Column names in each table must be unique. Column name 'PropertyCityAddress' in table 'Nashville' is specified more than once.

Total execution time: 00:00:00.065

In [61]:
ALTER TABLE Nashville
Add NewPropertyAddress NVARCHAR(255);

Commands completed successfully.

Total execution time: 00:00:00.146

In [66]:
UPDATE Nashville
set PropertyCityAddress = SUBSTRING(PropertyAddress,CHARINDEX(',',PropertyAddress)+2,LEN(PropertyAddress))

UPDATE Nashville
set NewPropertyAddress = SUBSTRING(PropertyAddress,1,CHARINDEX(',',PropertyAddress)-1)

select top (5) propertycityaddress, NewPropertyAddress
from Nashville

(56477 rows affected)

(56477 rows affected)

(5 rows affected)

Total execution time: 00:00:01.301

propertycityaddress,NewPropertyAddress
GOODLETTSVILLE,1808 FOX CHASE DR
GOODLETTSVILLE,1832 FOX CHASE DR
GOODLETTSVILLE,1864 FOX CHASE DR
GOODLETTSVILLE,1853 FOX CHASE DR
GOODLETTSVILLE,1829 FOX CHASE DR


4\. Substring the owner address

In [70]:
ALTER TABLE Nashville
Add OwnerCityAddress NVARCHAR(255), NewOwnerAddress NVARCHAR(255); OwnerStateAddress NVARCHAR(255)


Commands completed successfully.

Total execution time: 00:00:00.051

In [106]:
UPDATE Nashville
set OwnerStateAddress = RIGHT(OwnerAddress,2)

UPDATE Nashville
set NewOwnerAddress = SUBSTRING(OwnerAddress,1,CHARINDEX(',',OwnerAddress)-1)

-- you can use parsename for all of them, but note that it is in reverse order
UPDATE Nashville
set OwnerCityAddress = PARSENAME(REPLACE(OwnerAddress, ',', '.') , 2)




(56477 rows affected)

(56477 rows affected)

(56477 rows affected)

Total execution time: 00:00:02.604

4\. Update the data in soldasVacant for consistent

In [117]:
select soldasvacant, count(SoldAsVacant)
FROM Nashville
GROUP by SoldAsVacant
order by SoldAsVacant

UPDATE Nashville
set SoldAsVacant =  CASE WHEN SoldAsVacant = 'N' Then   'No'
                WHEN SoldAsVacant = 'Y' Then   'Yes'
            Else SoldAsVacant
            END


(2 rows affected)

(56477 rows affected)

Total execution time: 00:00:02.006

soldasvacant,(No column name)
No,51802
Yes,4675


: Msg 245, Level 16, State 1, Line 12
Conversion failed when converting the nvarchar value 'No' to data type bit.

In [118]:
-- check that no salesprice with null value
select SalePrice
from Nashville
where SalePrice is null

(0 rows affected)

Total execution time: 00:00:00.373

SalePrice


9\. Remove all duplicates

In [138]:
With RowCTE AS(
select *, ROW_NUMBER() OVER (PARTITION by parcelID,saleprice order by uniqueID) rownum
from Nashville
)

DELETE
from RowCTE
WHERE rownum>1





: Msg 102, Level 15, State 1, Line 11
Incorrect syntax near ','.

Total execution time: 00:00:00.095

10\. Remove redundant columns

In [142]:
alter TABLE nashville
drop column owneraddress

Commands completed successfully.

Total execution time: 00:00:00.340

In [146]:
alter TABLE nashville
drop column propertyaddress

: Msg 4924, Level 16, State 1, Line 1
ALTER TABLE DROP COLUMN failed because column 'propertyaddress' does not exist in table 'Nashville'.

Total execution time: 00:00:00.080

In [163]:
select Top(5) *
From Nashville



(5 rows affected)

Total execution time: 00:00:00.782

UniqueID,ParcelID,LandUse,SaleDate,SalePrice,LegalReference,SoldAsVacant,OwnerName,Acreage,TaxDistrict,LandValue,BuildingValue,TotalValue,YearBuilt,Bedrooms,FullBath,HalfBath,PropertyCityAddress,NewPropertyAddress,OwnerCityAddress,NewOwnerAddress,OwnerStateAddress
2045,007 00 0 125.00,SINGLE FAMILY,2013-04-09,240000.00,20130412-0036474,No,"FRAZIER, CYRENTHA LYNETTE",2.3,GENERAL SERVICES DISTRICT,50000,168200,235700,1986,3,3,0,GOODLETTSVILLE,1808 FOX CHASE DR,GOODLETTSVILLE,1808 FOX CHASE DR,TN
16918,007 00 0 130.00,SINGLE FAMILY,2014-06-10,366000.00,20140619-0053768,No,"BONER, CHARLES & LESLIE",3.5,GENERAL SERVICES DISTRICT,50000,264100,319000,1998,3,3,2,GOODLETTSVILLE,1832 FOX CHASE DR,GOODLETTSVILLE,1832 FOX CHASE DR,TN
54582,007 00 0 138.00,SINGLE FAMILY,2016-09-26,435000.00,20160927-0101718,No,"WILSON, JAMES E. & JOANNE",2.9,GENERAL SERVICES DISTRICT,50000,216200,298000,1987,4,3,0,GOODLETTSVILLE,1864 FOX CHASE DR,GOODLETTSVILLE,1864 FOX CHASE DR,TN
43070,007 00 0 143.00,SINGLE FAMILY,2016-01-29,255000.00,20160129-0008913,No,"BAKER, JAY K. & SUSAN E.",2.6,GENERAL SERVICES DISTRICT,50000,147300,197300,1985,3,3,0,GOODLETTSVILLE,1853 FOX CHASE DR,GOODLETTSVILLE,1853 FOX CHASE DR,TN
22714,007 00 0 149.00,SINGLE FAMILY,2014-10-10,278000.00,20141015-0095255,No,"POST, CHRISTOPHER M. & SAMANTHA C.",2,GENERAL SERVICES DISTRICT,50000,152300,202300,1984,4,3,0,GOODLETTSVILLE,1829 FOX CHASE DR,GOODLETTSVILLE,1829 FOX CHASE DR,TN
